In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.6.0


In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [5]:
#preprocess_sentence 함수 정의

#앞으로 사용할 모듈 임포트
# os: Operating System의 줄임말로, 운영체제에서 제공되는 여러 기능을 파이썬에서 사용할 수 있도록 함 (Ex. 디렉토리 경로 이동, 시스템 환경 변수 가져오기 등)
# re: Regular Expression의 줄임말로, 파이썬 정규표현식을 사용하기 위한 모듈
# numpy(NumPy): 행렬이나 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 라이브러리. 데이터 구조 외에도 수치 계산을 위해 효율적으로 구현된 기능을 제공
# tensorflow(TensorFlow): 데이터 흐름 프로그래밍을 위한 라이브러리. 데이터 그래프를 수치적으로 연산하기 위해 구글에서 만든 오픈 소스 라이브러리.
import os, re 
import numpy as np
import tensorflow as tf

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4   #[^a] a를 제외한 모든 문자
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 15: continue # 토큰의 개수가 15개를 넘어가는 문장 제외
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> and even though <end>',
 '<start> hallelujah <end>']

In [7]:
raw_corpus[:10]

["Now I've heard there was a secret chord",
 'That David played, and it pleased the Lord',
 "But you don't really care for music, do you?",
 'It goes like this',
 'The fourth, the fifth',
 'The minor fall, the major lift',
 'The baffled king composing Hallelujah Hallelujah',
 'Hallelujah',
 'Hallelujah',
 'Hallelujah Your faith was strong but you needed proof']

In [8]:
# 토큰의 개수가 아닌 센텐스 문자열자체 길이가 15초과인것을 continue 했다.
# 각 센텐스에 토큰의 개수(단어등 구분된 요소의 개수)가 15 개가 넘어가는 것을 제거하도록 변경


In [10]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
   
    preprocessed_sentence = preprocess_sentence(sentence)
    # 전처리된 센텐스의 토큰수가 15 이하인 것만 corpus 에 담음
    if len(preprocessed_sentence.split()) <= 15:
        corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [12]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    


from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=17 )


[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f10a06df280>


In [13]:
# token 단어 사전 확인 10개만
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [19]:
# 모델 학습시키기

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 128
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
#
dataset

## 인공지능 학습 시키기

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 64 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [20]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[ 8.86657290e-05,  3.54989133e-05,  3.08497292e-05, ...,
          8.59072679e-06, -2.48120596e-05,  7.79882175e-05],
        [ 1.06522828e-04,  2.69688862e-05,  8.37720381e-06, ...,
          1.95884670e-04, -1.09428554e-04,  8.61282169e-05],
        [ 2.18315836e-04, -2.17418983e-05, -3.91805697e-05, ...,
          2.57734879e-04, -2.71355122e-04,  5.39123757e-05],
        ...,
        [-6.94677350e-04,  3.58520367e-04, -7.73327949e-04, ...,
          2.00221912e-05, -3.98672681e-04, -3.20104795e-04],
        [-7.83167721e-04,  4.41129931e-04, -8.91265809e-04, ...,
         -4.57968417e-05, -4.52966196e-04, -5.07891178e-04],
        [-8.62936431e-04,  5.20330155e-04, -9.82404221e-04, ...,
         -1.06221552e-04, -5.14326384e-04, -6.79346442e-04]],

       [[ 8.86657290e-05,  3.54989133e-05,  3.08497292e-05, ...,
          8.59072679e-06, -2.48120596e-05,  7.79882175e-05],
        [ 2.40893161e-04,  7.30823522e-05, -3

In [21]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  768064    
_________________________________________________________________
lstm_2 (LSTM)                multiple                  4460544   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 25,922,337
Trainable params: 25,922,337
Non-trainable params: 0
_________________________________________________________________


In [22]:
# optimizer와 loss등 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

#Loss

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=10, validation_data=((enc_val, dec_val))) # 만들어둔 데이터셋으로 모델을 학습한다. 10번 학습을 반복하겠다는 의미다.
# val_loss:2.0111

Epoch 1/10
1218/1218 [==============================] - 119s 96ms/step - loss: 3.3229 - val_loss: 3.0253
Epoch 2/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.9538 - val_loss: 2.8356
Epoch 3/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.7893 - val_loss: 2.6841
Epoch 4/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.6548 - val_loss: 2.5564
Epoch 5/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.5385 - val_loss: 2.4442
Epoch 6/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.4351 - val_loss: 2.3442
Epoch 7/10
1218/1218 [==============================] - 118s 96ms/step - loss: 2.3401 - val_loss: 2.2531
Epoch 8/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.2528 - val_loss: 2.1680
Epoch 9/10
1218/1218 [==============================] - 117s 96ms/step - loss: 2.1709 - val_loss: 2.0865
Epoch 10/10
1218/1218 [==============================] 

In [23]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [25]:
lyricist = model
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

# 회고
- 자연어 처리 과정을 통해 문장이 sequantial 하게 연결 (문법과 단어의 상관관계등에 따라) 되어 있다는 점을 다시한번 인지할 수 있었다.
- 위의 특성에 착안하여, 빅데이터를 통한 통계를 바탕으로 자연스러운 문장을 생성하려고 하는 발상이 참신했다.
- embedding의 의미와 embedding 깊이조절을 통한 벡터공간에서 단어의 추상적 표현을 설정하고 학습하여 단어간 상관관계를 유추
- 직접 모델을 설계하고 테스트와 밸리데이션을 나누어 평가하여 보면서 작동을 이해해보았다.

지난시간의 CNN과 RNN 모델을 살펴본 결과
지속적으로 의존하여 사용한다고 하였을때,
인공지능이 위와 같은 방식으로 데이터를 통계적으로 접근한다면 
(통계적으로 확률이 높은 결과 또는 다수결로 가져가면) 지성이 무뎌지는 결과가 발생할 수 있다.
- 전제가 통계와 다수결 결정이 옳은 방향으로 진행되었다를 가정하였기 때문에
1. 모집단 자체가 잘못 된 방향으로 설계가 되어있다면 애초에 바른 값이 나올 수 없다. 
그러나 집단지성과 시간에 따라 축적된 지식 데이터 (빅데이터)의 가리키는 방향의 값이 많은 시도를 통해 안정화되었고 회귀되었다고 볼 수 있으므로, 현재시점(prediction을 통해 결과값 도출을 시도하는 시점)에서는 가장 알맞은 값을 예측할 수 있다.
2. 새로운(잘못되지않은, 모집단에서 벗어난) 결과가 나올 수 없다.
새로운발견이나 지성 변화는 대중적으로 사용되지 않거나.
소수의 사람이 알고 사용할 경우가 높다.
패러다임의 변환시기나 노벨티 있는 사용 
혁신이나 대격변이 반영 되지 않아. 기존의 지성(지능)에 갖힐수 밖에 없다.
3. 빅데이터를 사용한 딥러닝을 통해 미쳐 인간의 사고능력으로 도출되지 못한 요소들간의 상관관계와 피쳐를 뽑아 내고 그로인해 결과를 도출할 수 있으나 한계가 있다. 


In [26]:
# 추가 아무 문장이나 생성해보았다.
generate_text(lyricist, tokenizer, init_sentence="<start> ", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 아무 문장을 생성

'<start> i m not living right now <end> '